# Welcome to ExKaldi

In this section, we will decode the test data based on HMM-DNN model and WFST graph.

In [1]:
CONDA_DIR = "/home/khanh/workspace/miniconda3"
KALDI_ENV = "kaldi"
EXKALDI_ENV = "exkaldi"
KALDI_ROOT = "/home/khanh/workspace/projects/kaldi"

DATA_DIR = "librispeech_dummy"

def import_exkaldi():
    import os

    # add lib path
    os.environ["LD_LIBRARY_PATH"] = ";".join([
        os.path.join(CONDA_DIR, "envs", KALDI_ENV, "lib"),
        os.path.join(CONDA_DIR, "envs", EXKALDI_ENV, "lib"),
    ])

    import exkaldi
    exkaldi.info.reset_kaldi_root(KALDI_ROOT)

    return exkaldi
exkaldi = import_exkaldi()
dataDir = "librispeech_dummy"

import os

exkaldi.info.reset_kaldi_root( yourPath )
If not, ERROR will occur when implementing some core functions.


Restorage the posteiori probability of AM from file (generated in 11_train_DNN_acoustic_model_with_tensorflow).

In [2]:
probFile = os.path.join(dataDir, "exp", "train_DNN", "amp.npy")

prob = exkaldi.load_prob(probFile)

prob

In [3]:
prob.subset(nHead=1).data

{'1272-128104-0000': array([[ 9.464016  , -2.0549624 , -0.79543144, ...,  3.4590974 ,
         -4.529152  ,  4.7647963 ],
        [ 9.37714   , -1.2627714 , -0.8936229 , ...,  3.8034425 ,
         -3.8769655 ,  4.1946864 ],
        [10.027403  , -0.81561977, -0.88878584, ...,  3.2106102 ,
         -3.2808084 ,  3.7135477 ],
        ...,
        [10.073996  , -1.0323883 , -0.8198512 , ...,  1.5600028 ,
         -1.2490301 ,  5.2349787 ],
        [10.578005  , -2.0080304 , -0.48259932, ...,  1.3785863 ,
         -0.68345255,  4.589739  ],
        [10.7762165 , -2.7672167 , -0.40990275, ...,  0.7246169 ,
         -1.2750771 ,  3.3913956 ]], dtype=float32)}

As above, this is naive output without log softmax activation function. We need do softmax.

Exkaldi Numpy achivements have a method __.map(...)__ . We map a softmax function to all matrixs.

In [4]:
prob = prob.map( lambda x: exkaldi.nn.log_softmax(x, axis=1) )

prob.subset(nHead=1).data

{'1272-128104-0000': array([[ -3.0525103, -14.571488 , -13.311957 , ...,  -9.057428 ,
         -17.045677 ,  -7.75173  ],
        [ -2.7164993, -13.356411 , -12.987263 , ...,  -8.290197 ,
         -15.970605 ,  -7.898953 ],
        [ -2.2598648, -13.102887 , -13.176054 , ...,  -9.076657 ,
         -15.568076 ,  -8.57372  ],
        ...,
        [ -1.4313774, -12.537762 , -12.325224 , ...,  -9.94537  ,
         -12.754403 ,  -6.2703943],
        [ -1.9326248, -14.51866  , -12.993229 , ..., -11.132044 ,
         -13.194082 ,  -7.920891 ],
        [ -1.8406677, -15.384101 , -13.026787 , ..., -11.892267 ,
         -13.891961 ,  -9.225489 ]], dtype=float32)}

Then decode based on WFST. HCLG graph file and HMM model file have been generated (07_train_triphone_HMM-GMM_delta).

In [5]:
HCLGFile = os.path.join(dataDir, "exp", "train_delta", "graph", "HCLG.fst")

hmmFile = os.path.join(dataDir, "exp", "train_delta", "final.mdl")

And for convenience, prepare lexicons.

In [6]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.decode.graph.load_lex(lexFile)

lexicons

Use __nn_decode__ function. 

In [7]:
lat = exkaldi.decode.wfst.nn_decode(prob, hmmFile, HCLGFile, symbolTable=lexicons("words"))

lat

In [8]:
outDir = os.path.join(dataDir, "exp", "train_DNN", "decode_test")

exkaldi.utils.make_dependent_dirs(outDir, False)

lat.save( os.path.join(outDir,"test.lat") )

'librispeech_dummy/exp/train_DNN/decode_test/test.lat'

From lattice get 1-best result and score it.

In [9]:
refIntFile = os.path.join(dataDir, "exp", "train_delta", "decode_test", "text.int")

for penalty in [0., 0.5, 1.0]:
    for LMWT in range(10, 15):
    
        newLat = lat.add_penalty(penalty)
        result = newLat.get_1best(lexicons("words"), hmmFile, lmwt=LMWT, acwt=0.5)

        score = exkaldi.decode.score.wer(ref=refIntFile, hyp=result, mode="present")
        
        print(f"Penalty {penalty}, LMWT {LMWT}: WER {score.WER}")

Penalty 0.0, LMWT 10: WER 108.83
Penalty 0.0, LMWT 11: WER 108.83
Penalty 0.0, LMWT 12: WER 108.83
Penalty 0.0, LMWT 13: WER 108.83
Penalty 0.0, LMWT 14: WER 108.83
Penalty 0.5, LMWT 10: WER 108.83
Penalty 0.5, LMWT 11: WER 108.83
Penalty 0.5, LMWT 12: WER 108.83
Penalty 0.5, LMWT 13: WER 108.83
Penalty 0.5, LMWT 14: WER 108.83
Penalty 1.0, LMWT 10: WER 108.35
Penalty 1.0, LMWT 11: WER 108.35
Penalty 1.0, LMWT 12: WER 108.35
Penalty 1.0, LMWT 13: WER 108.35
Penalty 1.0, LMWT 14: WER 108.35


In step 10_process_lattice_and_score, the best WER based on HMM-GMM model is about 135% and here is 107% in our experiment. So it is a truth that the performance of HMM-DNN got better.

Up to here, the simple tutorial is over.